<a href="https://colab.research.google.com/github/ighoshsubho/youtube-QA-summarize/blob/main/youtube_summarizer_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%time

! pip install -qq -U langchain youtube_transcript_api einops
! pip install -qq -U accelerate bitsandbytes xformers

In [ ]:
!pip install transformers

In [ ]:
from IPython.display import YouTubeVideo

from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
from langchain.chains.summarize import load_summarize_chain
from langchain.llms import HuggingFacePipeline

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

import torch

import langchain
print(langchain.__version__)

0.0.268


In [ ]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=tAuRQs_d9F8&t=52s")
transcript = loader.load()

In [ ]:
transcript

[Document(page_content="I heard you liked factories so I made you a factory inside a factory which inherits from an abstract Factory so it can create new factories but enough about programming in Java in this video we will learn about eight design patterns every developer should know in 1994 the gang of four released the holy book design patterns introducing 23 object-oriented design patterns falling into one of three buckets creational patterns structural patterns and behavioral patterns while some argue that it stated the fact that a 30 year old book is still being discussed definitely means something especially in a world where JavaScript Frameworks are going out of style faster than you can say JavaScript was a mistake anyways let's start with our first creational pattern the factory imagine that you want a burger but you don't want to have to worry about getting all the ingredients and putting them together so instead you just order a burger well we can do the same thing with code

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=50)
texts = text_splitter.split_documents(transcript)

In [ ]:
%%time

model_repo = 'bigscience/bloom-560m'

tokenizer = AutoTokenizer.from_pretrained(model_repo)

model = AutoModelForCausalLM.from_pretrained(model_repo,
                                             load_in_8bit=True,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             low_cpu_mem_usage=True,
                                             trust_remote_code=True
                                            )
max_len = 2048 # 1024
task = "text-generation"
T = 0

Some weights of BloomForCausalLM were not initialized from the model checkpoint at bigscience/bloom-560m and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CPU times: user 26.7 s, sys: 3.62 s, total: 30.3 s
Wall time: 52.3 s


In [ ]:
pipe = pipeline(
    task=task,
    model=model,
    tokenizer=tokenizer,
    max_length=max_len,
    temperature=T,
    top_p=0.95,
    repetition_penalty=1.15,
    pad_token_id = 11
)

llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=True)

In [ ]:
chain

MapReduceDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=True, tags=None, metadata=None, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=True, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:', template_format='f-string', validate_template=True), llm=HuggingFacePipeline(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7c885da73580>, model_id='gpt2', model_kwargs=None, pipeline_kwargs=None), output_key='text', output_parser=StrOutputParser(), return_final_only=True, llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=True, tags=None

In [ ]:
### default prompt template
chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [ ]:
%%time

# Run the chain with verbose=True
summary = chain.run(texts)
summary



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"I heard you liked factories so I made you a factory inside a factory which inherits from an abstract Factory so it can create new factories but enough about programming in Java in this video we will learn about eight design patterns every developer should know in 1994 the gang of four released the holy book design patterns introducing 23 object-oriented design patterns falling into one of three buckets creational patterns structural patterns and behavioral patterns while some argue that it stated the fact that a 30 year old book is still being discussed definitely means something especially in a world where JavaScript Frameworks are going out of style faster than you can say JavaScript was a mistake anyways let's start with our first creational pattern the factory imagine that you want a burger but you don't want to have to worry abo

Token indices sequence length is longer than the specified maximum sequence length for this model (5664 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1938 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2427 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5216 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3125 > 1024). Running this sequence through the model will result in indexing errors




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"

1. The concept behind these designs has been very simple, they were designed by people who had no clue as much time spent on designing software.
2. They didn't require many changes when compared other languages such as:
Java, C# etc.. 3.
They did not need special tools (such : IDE) 4.

A:

(I am using JDK 1)
The following example shows why your class doesn't work properly. 
public interface Furniture {
    void Add(FancyBread b)
    public static bool IsAddableToThisClass();

    // This function returns true iff there exists another instance.
    private Boolean HasOtherInstance = false; 
    
    /**
     * Returns false.
     */ 
    /// <summary>Returns null otherwise</summary>    
    [System]   
        protected virtual Object GetObjectByID<T>()
        {return null;}  

        #region Constructors & Destructor
        
       

'\n\nThe following code snippet creates three subscription records. The first record has status = "Active";\nthe second, third are not active.\nIn order to: create any other type (evaluated) subscribe on another group I would like my method as follows:  \n\npublic void CreateSubscription(string id)\n{\n    Subscription s=new Subscriptions();\n\n    // Add some data here.\n    var item=SubscriberFactory\n       .CreateItemFromUserIdAndName(\n            Id,\n            Name\n        );//This line adds items from User ID/name field\n        foreach((var x : NewSubItems)\n        {\n            try \n            //to make sure there isn\'t anything wrong.\n            ConsoleWriter\n                <<x\n                   .AddToList()\n               .SendAsync(\n                    MessageTypeEnum\n                    => \n                            (int)(DateTimeOffset).Now - DateTimeSpan\n                       .Parse(\n                            Convert\n                           

In [ ]:
import locale

locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
! pip install transformers

In [ ]:
from transformers import pipeline
model_checkpoint = "IProject-10/bert-base-uncased-finetuned-squad2"
question_answerer = pipeline("question-answering", model=model_checkpoint)

In [ ]:
text1 = """{}""".format(transcript[0])[14:]
text1

'I heard you liked factories so I made you a factory inside a factory which inherits from an abstract Factory so it can create new factories but enough about programming in Java in this video we will learn about eight design patterns every developer should know in 1994 the gang of four released the holy book design patterns introducing 23 object-oriented design patterns falling into one of three buckets creational patterns structural patterns and behavioral patterns while some argue that it stated the fact that a 30 year old book is still being discussed definitely means something especially in a world where JavaScript Frameworks are going out of style faster than you can say JavaScript was a mistake anyways let\'s start with our first creational pattern the factory imagine that you want a burger but you don\'t want to have to worry about getting all the ingredients and putting them together so instead you just order a burger well we can do the same thing with code if it takes a list o

In [ ]:
context = text1
context

'I heard you liked factories so I made you a factory inside a factory which inherits from an abstract Factory so it can create new factories but enough about programming in Java in this video we will learn about eight design patterns every developer should know in 1994 the gang of four released the holy book design patterns introducing 23 object-oriented design patterns falling into one of three buckets creational patterns structural patterns and behavioral patterns while some argue that it stated the fact that a 30 year old book is still being discussed definitely means something especially in a world where JavaScript Frameworks are going out of style faster than you can say JavaScript was a mistake anyways let\'s start with our first creational pattern the factory imagine that you want a burger but you don\'t want to have to worry about getting all the ingredients and putting them together so instead you just order a burger well we can do the same thing with code if it takes a list o

In [ ]:
question = "What are 23 object-oriented design patterns"
question_answerer(question=question, context=context)

{'score': 0.7022665143013,
 'start': 426,
 'end': 469,
 'answer': 'structural patterns and behavioral patterns'}

In [ ]:
!pip install gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 7.2 MB/s eta 0:00:00


In [ ]:
import gradio as gr
index = None

In [ ]:
def build_the_bot(input_text):
  loader = YoutubeLoader.from_youtube_url("{}".format(input_text))
  transcript = loader.load()
  texts = text_splitter.split_documents(transcript)
  text1 = """{}""".format(transcript[0])[14:]
  context = text1
  return('Bot Build Successfull!!!')

In [ ]:
def build_the_bot2(input_text):
  loader = YoutubeLoader.from_youtube_url("{}".format(input_text))
  transcript = loader.load()
  texts = text_splitter.split_documents(transcript)
  text1 = """{}""".format(transcript[0])[14:]
  context = text1
  return('Summary goes here!')

In [ ]:
def chat(chat_history, user_input):

  output = question_answerer(question=user_input, context=context)
  bot_response = output["answer"]
  #print(bot_response)
  response = ""
  for letter in ''.join(bot_response): #[bot_response[i:i+1] for i in range(0, len(bot_response), 1)]:
      response += letter + ""
      yield chat_history + [(user_input, response)]

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown('# YouTube Q&A and Summarizer Bot')
    with gr.Tab("Input URL of video you wanna load -"):
        text_input = gr.Textbox()
        text_output = gr.Textbox()
        text_button1 = gr.Button("Build the Bot!!!")
        text_button1.click(build_the_bot, text_input, text_output)
        text_button2 = gr.Button("Summarize...")
        text_button2.click(build_the_bot2, text_input, text_output)
    with gr.Tab("Knowledge Base -"):
#          inputbox = gr.Textbox("Input your text to build a Q&A Bot here.....")
          chatbot = gr.Chatbot()
          message = gr.Textbox ("What is this document about?")
          message.submit(chat, [chatbot, message], chatbot)

demo.queue().launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e0378c69f28a520598.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
